<h1>Titanic - Machine Learning from disaster</h1>
<img src="https://www.kaggle.com/competitions/3136/images/header"/>
<ul>
    <li>Vamos utilizar os dados do Kaggle</li>
    <ul>
        <li>é um dataset de <strong>competição</strong></li>
        <li>o resultado é avaliado em <strong>precisão</strong>:</li>
            <ul>
                <li><em>"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como precisão."</em></li>
            </ul>
    </ul>
</ul>

<h2>Començando a entender a base</h2>
<ul>
    <li>Podemos utilizar o ydata-profiling para começar a entender essa base.</li>
    <ul>
        <li><a href="https://github.com/ydataai/ydata-profiling">https://github.com/ydataai/ydata-profiling</a></li>
    </ul>
</ul>

In [14]:
# importando o pandas
import pandas as pd

In [15]:
treino = pd.read_csv(f'./train.csv')
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
# Importando o ProfileReport do ydata-profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(treino, title="titanic_treino")

In [17]:
# criando relatório
profile.to_file(f'./titanic_treino.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 65.22it/s]


In [18]:
# verificando as informações da base
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [19]:
# verificando os tipos de dados 
treino.dtypes.value_counts()

int64      5
object     5
float64    2
Name: count, dtype: int64

In [20]:
# E os valores nulos
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
dtype: int64

In [21]:
# Visualizando a base de teste
teste = pd.read_csv(f'./test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [22]:
# Verificando as informações da base
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [23]:
teste.isnull().sum().sort_values(ascending=False).head(5)

Cabin     327
Age        86
Fare        1
Name        0
Pclass      0
dtype: int64

<ul>
    <li>Temos colunas que <strong>possuem valores vazios na base de teste que não estão vazios na base de treino</strong> (nesse caso, precisariamos tratar essas colunas apenas na base de teste). </li>
</ul>

<h3>Realizando os tratamentos iniciais dos dados</h3>
<ul>
    <li>Podemos começar tratando os valores vazios</li>
</ul>

In [24]:
# verificando novamente valores vazios na base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
dtype: int64

In [25]:
# retornando o shape da base
treino.shape

(891, 12)

In [26]:
# visualizando novamente a base
treino.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [ ]:
# verificando a cardinalidade dos dados
treino.nunique().sort_values(ascending=False )

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Embarked         3
Pclass           3
Survived         2
Sex              2
dtype: int64

<ul>
    <li>colunas com <strong>alta cardinalidade</strong> como nome, número do ticket e até mesmo o código da cabine não vão ajudar o modelo pois podem torná-lo <strong>menos generalizável</strong>. Nesse caso, vamos <strong>eliminar essas colunas</strong></li>
</ul>

In [28]:
# Eliminando para a base de treino
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [29]:
# Eliminando para a base de teste
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)

<ul>
    <li>Agora precisamos tratar a coluna <strong>Age</strong>, que possui valores tanto no treino quanto no teste</li>
    <ul>
        <li>Como essa coluna possui a idade dos passageiros, podemos utilizar a <strong>média das idades</strong> para substituir os valores vazios</li>
    </ul>
</ul>


In [31]:
# verificando a média das idades para a base de treino
treino.Age.mean()

np.float64(29.69911764705882)

In [32]:
# Podemos agora selecionar as colunas com idade vazia e substituir por essa média
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()

In [33]:
# mesma coisa para teste
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()

<ul>
    <li>Para a base de treino, ainda temos 2 valores vazios na coluna <strong>Embarked</strong></li>
    <ul>
        <li>Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela <strong>moda</strong></li>
    </ul>
</ul>

In [34]:
# verificando a coluna embarked
treino.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [35]:
# selecionando a moda da coluna
treino.Embarked.mode()[0]

'S'

In [36]:
# substituindo os valores vazios
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

<ul>
    <li>Trabalhando com o valor de <strong>Fare</strong></li>
    <ul>
        <li>Podemos usar a mesma logica de substituição pela <strong>média</strong></li>
    <ul>
</ul>

In [37]:
# verificando a media da coluna Fare para a base de teste
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

- verificando novamente os valores vazios

In [38]:
# paraa base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [39]:
# paraa base de teste
teste.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

### considerando apenas as colunas que não são texto

In [ ]:
# colunas de texto na base de treino
col_treino_nr = treino.columns[treino.dtypes != 'object']

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [ ]:
# selecionando apenas os valores numéricos da base de treino
treino_nr = treino.loc[:, col_treino_nr]

In [43]:
# para a base de teste
col_teste_nr = teste.columns[teste.dtypes != 'object']

In [45]:
# valores numéricos da base de teste
teste_nr = teste.loc[:, col_teste_nr]

## Podemos agora selecionar um modelo para classificar esses dados
* Para começar, vamos testar entre:
    * **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html
    
    * **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
    * **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

* Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    * Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 

In [46]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [48]:
# Separando a base de treino em X e Y
X = treino_nr.drop(['PassengerId', 'Survived'], axis=1)
y = treino.Survived

In [49]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.33, random_state=42)

- Para a **árvore de classificação**

In [55]:
# Fazendo a importação
from sklearn import tree

In [56]:
# criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [57]:
# fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train, y_train)

In [58]:
# fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para **KNeighborsClassifier**

In [ ]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# fit com os dados
clf_knn = clf_knn.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para **Regressão Linear**

In [ ]:
# importando
from sklearn.linear_model import LogisticRegression

In [ ]:
# criando o classificador
clf_rl = LogisticRegression(random_state=42)

In [ ]:
# fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

In [ ]:
# fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)